In [1]:
from unified_planning.shortcuts import *
from unified_planning.plans import *
from importlib import reload

In [3]:
import unifiedplanning_blocksworld as bw
import prompts
import planbench as pb
import policy_model as policy
import llm_utils
import data_loader
import pandas as pd
import regex as re

In [50]:
reload(policy)
reload(prompts)
reload(bw)
reload(pb)
reload(llm_utils)
reload(data_loader)

<module 'data_loader' from '/srv/chawak/planning-with-llms/src/data_loader.py'>

In [37]:
llm_utils.query_llm('help')

"I'd be happy to help you. Please provide more details about what you need help with, such as:\n\n1. A specific problem you're facing\n2. A question you have\n3. A topic you'd like to discuss\n4. A task you need assistance with\n\nYou can also type one of the following commands to get started:\n\n* `general` for general conversation\n* `math` for math-related help\n* `programming` for programming-related help\n* `writing` for writing-related help\n* `other` for anything else\n\nRemember, I'm here to help and provide guidance to the best of my abilities!"

In [4]:
problem = bw.BlocksworldProblem()

In [5]:
pb.parse_planbench_initial_condition(problem, prompts.nextaction_problem_init)
pb.parse_planbench_goal_state(problem, prompts.nextaction_problem_goal)
#print(problem.initial_values)
#initial vals ko vaapais readable mein translate karna hoga

{clear(blue): true, clear(yellow): true, hand_empty: true, on(blue, orange): true, on(orange, red): true, on(blue, blue): false, on(yellow, blue): false, on(orange, blue): false, on(red, blue): false, on(blue, yellow): false, on(yellow, yellow): false, on(orange, yellow): false, on(red, yellow): false, on(yellow, orange): false, on(orange, orange): false, on(red, orange): false, on(blue, red): false, on(yellow, red): false, on(red, red): false, clear(orange): false, clear(red): false, holding(blue): false, holding(yellow): false, holding(orange): false, holding(red): false}


In [ ]:

pi = policy.PolicyModel()
    # def next_action_one_shot(self, 
    #                          problem_init, problem_goal,problem_action_hist,  # the state S
    #                          example_init, example_goal,example_action_hist,example_next_action)
plan=[]
#action_tuple = pi.next_action_one_shot(prompts.nextaction_problem_init, prompts.nextaction_problem_goal,
#                                       plan,
#                                       prompts.nextaction_example_init, prompts.nextaction_example_goal,
#                                       prompts.nextaction_example_action_hist,prompts.nextaction_example_nextaction)


In [82]:
action_tuple=('unstack','blue','orange')
#print(action_tuple)
next_action=problem.create_action(action=action_tuple[0],blocks=action_tuple[1:])
#print(next_action)

('unstack', 'blue', 'orange')
('blue', 'orange')
unstack(blue, orange)


In [83]:
simulation=problem.create_seq_simulation()
current_state=simulation.get_initial_state()
new_state=problem.validate_action(current_state,next_action,simulation)
#if(new_state):
    #print(new_state)
#else:
    #print('Invalid action proposed by policy')

{on(blue, orange): false, holding(blue): true, clear(orange): true, hand_empty: false, clear(blue): true, clear(yellow): true, on(orange, red): true, on(blue, blue): false, on(yellow, blue): false, on(orange, blue): false, on(red, blue): false, on(blue, yellow): false, on(yellow, yellow): false, on(orange, yellow): false, on(red, yellow): false, on(yellow, orange): false, on(orange, orange): false, on(red, orange): false, on(blue, red): false, on(yellow, red): false, on(red, red): false, clear(red): false, holding(yellow): false, holding(orange): false, holding(red): false}


In [6]:
reload(data_loader)
df=data_loader.load_data("../data/train.parquet")
df.head()


,task,prompt_type,domain,instance_id,example_instance_ids,query,ground_truth_plan
885,task_1_plan_generation,oneshot,blocksworld_3,2,[1],I am playing with a set of blocks where I need...,(pick-up red)\n(stack red orange)\n(pick-up bl...
886,task_1_plan_generation,oneshot,blocksworld_3,3,[2],I am playing with a set of blocks where I need...,(unstack blue orange)\n(put-down blue)\n(unsta...
887,task_1_plan_generation,oneshot,blocksworld_3,4,[3],I am playing with a set of blocks where I need...,(unstack red blue)\n(put-down red)\n(unstack b...
888,task_1_plan_generation,oneshot,blocksworld_3,5,[4],I am playing with a set of blocks where I need...,(unstack red blue)\n(put-down red)\n(unstack b...
889,task_1_plan_generation,oneshot,blocksworld_3,6,[5],I am playing with a set of blocks where I need...,(unstack red orange)\n(put-down red)\n(unstack...


In [156]:
#df.size

4200

In [5]:
def evaluate(attempts,struct,diff_len,results,actions_to_goal,gplan_lens,valid_action_count):
    metrics=pd.DataFrame({
        'Parsable':struct,
        'Attempts':attempts,
        'Terminated':results,
        'Valid Actions':valid_action_count,
        'Distance from goal':actions_to_goal,
        'Diff in plan len':diff_len,
        'GOLD Plan len':gplan_lens
    })
    correct_plans=sum(results)
    c_rate=correct_plans/len(results)
    c_rate=format(c_rate,".2f")
    return metrics,c_rate

In [130]:
#vanilla method
from collections import defaultdict
import math
def main_fullSol(df,model,temp):
    results=[False]*len(df)
    well_struct=[False]*len(df)
    num_tries=[0]*len(df)
    diff_planlen=[-math.inf]*len(df)
    actions_to_goal=[-math.inf]*len(df)
    gplan_lens=[-math.inf]*len(df)
    valid_action_count=[0]*len(df)
    for i in range(0,len(df)):
    #for i in range(0,1):
        
        print(f'\n\n--------------------Entering main loop PROBLEM #{i}--------------------')
        
        #declarations: Blocksworld Problem object
        problem = bw.BlocksworldProblem()
        model_plan=None

        #create prompt using PLANBENCH dataset
        #prompt=prompts.make_prompt(df.iloc[i]['query'])
        #print(f'\n\nMake prompt:{prompt} \nEND Make prompt')

        #create prompt using OUR dataset
        prompt,init,goal=prompts.make_prompt(df.iloc[i]['init'],df.iloc[i]['goal'],df.iloc[i]['demo_init'],df.iloc[i]['demo_goal'],df.iloc[i]['demo_plan'])

        #parse PLANBENCH dataset into Blocksworld object
        #pbstring=prompts.get_pbstring(df.iloc[i]['query'])
        #init,goal=pb.parse_problem(pbstring)
        #pb.parse_planbench_initial_condition(problem, init)
        #pb.parse_planbench_goal_state(problem, goal)
        #print(f'\n\nBlocksworld Problem Initial Values:{problem.initial_values}')
        #print(f'\nBlocksworld Problem Goal Condition:{problem.goals}')
        
        #changed generation of Blocksworld object for OUR dataset
        pb.parse_planbench_initial_condition(problem, init)
        pb.parse_planbench_goal_state(problem, goal)
        print(f'\n\nBlocksworld Problem Initial Values:{problem.initial_values}')
        print(f'\nBlocksworld Problem Goal Condition:{problem.goals}')

        #activate solve mode
        pi = policy.PolicyModel()
        #query the llm, parse the actions, return tuples
        tags="Answer within the [PLAN] [PLAN END] tags."
        #tags="Use the first solved example [STATEMENT] to form a plan for the second [STATEMENT]. Answer within the [PLAN] [PLAN END] tags."
        response=pi.Vanilla_fullSol_one_shot(prompt+"\n" + tags,model=model,temp=temp)
        actions=response[0]
        tries=response[1]

        num_tries[i]=tries

        if not actions:
            well_struct[i]=False
            print(f'\n\n LLM exceeded 3 tries')
            results[i]=False
            continue
        well_struct[i]=True
        plan_len=len(actions)
        
        #generate plan from proposed actions
        #actions=[('unstack', 'brown', 'teal'), ('put-down', 'brown'), ('unstack', 'teal', 'violet'), ('put-down', 'teal'), ('unstack', 'violet', 'teal'), ('put-down', 'violet')] 
        #actions=[('unstack', 'green', 'pink'), ('put-down', 'green'), ('unstack', 'pink', 'red'), ('put-down', 'pink'),('pick_up', 'green'), ('stack', 'green' , 'red')] 
        #plan_len=len(actions)

        print(f'\n\nProposed actions by LLM : {actions} \n  END')
        model_plan=problem.create_plan_from_tuples(actions)
        if not model_plan:
            results[i]=False
            print('!!!!!!!!!!!Action statement NOT FROM LIST OF ALLOWED actions!!!!!!!!!!!!')
            #continue

        if model_plan:
            print(f'LLM PLAN: {model_plan}')

        #validate and apply    
        simulation=problem.create_seq_simulation()
        va_counter=0
        valid_state,flag,va_counter=problem.check_and_apply(simulation,model_plan)
        valid_action_count[i]=va_counter
        if not flag:
            results[i]=False
            #continue
            

        #compute actions to goal from last valid state
        # print(f'UBS OBJECT FOR LAST VALID STATE: {valid_state,sim}')
        a2g=problem.actions_to_goal(valid_state)
        actions_to_goal[i]=a2g

        
        #get gold-plan on problem
        gold_plan=problem.generate_plan()
        gplan_actions=str(gold_plan.plan).strip().split('\n')
        gplan_actions=gplan_actions[1:]
        gplan_len=len(gplan_actions)
        gplan_lens[i]=gplan_len
        print(f'\n\n$$$Proposed actions by GOLD PLAN $$$ : {gplan_actions} \n END')


        #termination-check 
        if problem.terminate(valid_state):
            # print(f'Check and Apply return STATE:{valid_state}')
            # print(f'v/s')
            # print(f'GOAL STATE: {problem.goals}')
            # print('Setting result as true')
            results[i]=True
            diff_planlen[i]=plan_len-gplan_len
            continue
        
        results[i]=False

        #compute metrics

    metrics,c_rate=evaluate(num_tries,well_struct,diff_planlen,results,actions_to_goal,gplan_lens,valid_action_count)
    return metrics,c_rate

The history saving thread hit an unexpected error (OperationalError('database or disk is full')).History will not be written to the database.


In [66]:
#load data for PLANBENCH dataset
#df=data_loader.load_data("../data/train.parquet")
#df.head()
#results=main_fullSol(df)

In [67]:
#print(f"\n\n Model solution's verdict: {results}")

In [68]:
#df.head

In [131]:
#load data for OUR dataset
import pandas as pd
n=3
df=pd.read_csv(f'../data/{n}_blocks/Oneshot_val_{n}_blocks')
df.drop(columns='Unnamed: 0',inplace=True)
df

,init,goal,demo_init,demo_goal,demo_plan
0,"(('violet', 'teal', 'brown'),)","(('violet',), ('teal',), ('brown',))","the green block is clear, the hand is empty, ...","the green block is on the table, the red bloc...",[PLAN]\nunstack the green block from on top of...
1,"(('red', 'pink', 'green'),)","(('pink',), ('red', 'green'))","the pink block is clear, the red block is clea...","the red block is on the table, the pink block...",[PLAN]\nunstack the pink block from on top of ...
2,"(('pink', 'red', 'green'),)","(('red', 'pink'), ('green',))","the violet block is clear, the hand is empty,...","the teal block is on the table, the violet bl...",[PLAN]\nunstack the violet block from on top o...
3,"(('teal', 'violet', 'brown'),)","(('violet',), ('teal',), ('brown',))","the brown block is clear, the hand is empty, ...","the violet block is on the table, the teal bl...",[PLAN]\nunstack the brown block from on top of...
4,"(('pink',), ('green', 'red'))","(('pink',), ('red', 'green'))","the brown block is clear, the hand is empty, ...","the brown block is on the table, the violet b...",[PLAN]\nunstack the brown block from on top of...
5,"(('pink', 'green'), ('red',))","(('red', 'pink'), ('green',))","the violet block is clear, the brown block is ...","the violet block is on the table, the brown b...",[PLAN]\nunstack the violet block from on top o...
6,"(('violet',), ('teal',), ('brown',))","(('brown', 'teal', 'violet'),)","the pink block is clear, the red block is clea...","the pink block is on the table, the red block...",[PLAN]\nunstack the pink block from on top of ...
7,"(('brown', 'violet'), ('teal',))","(('teal', 'brown', 'violet'),)","the pink block is clear, the green block is cl...","the green block is on the table, the red bloc...",[PLAN]\nunstack the green block from on top of...
8,"(('pink',), ('red', 'green'))","(('green', 'pink'), ('red',))","the red block is clear, the green block is cle...","the green block is on the table, the pink blo...",[PLAN]\nunstack the red block from on top of t...
9,"(('green', 'pink'), ('red',))","(('red', 'pink'), ('green',))","the violet block is clear, the teal block is c...","the violet block is on the table, the brown b...",[PLAN]\nunstack the teal block from on top of ...


In [43]:
# df=df[1:2]
# df

In [44]:
df.shape

(15, 5)

In [132]:
model='google/gemma-2-9b-it'
temp=0

In [133]:
response=main_fullSol(df,model=model,temp=temp)



--------------------Entering main loop PROBLEM #0--------------------
State in create_ic (('violet', 'teal', 'brown'),)
State in create_goal (('violet',), ('teal',), ('brown',))


Blocksworld Problem Initial Values:{clear(brown): true, hand_empty: true, on_table(violet): true, on(teal, violet): true, on(brown, teal): true, on(brown, brown): false, on(violet, brown): false, on(teal, brown): false, on(brown, violet): false, on(violet, violet): false, on(violet, teal): false, on(teal, teal): false, clear(violet): false, clear(teal): false, on_table(brown): false, on_table(teal): false, holding(brown): false, holding(violet): false, holding(teal): false}

Blocksworld Problem Goal Condition:[on_table(violet), on_table(teal), on_table(brown)]
*************** PROMPT ***************: 

 I am playing with a set of blocks where I need to arrange the blocks into stacks
        Here are the actions I can do: Pick up a block, Unstack a block from on top of another block, Put down a block, Stack a

In [126]:
metrics_df=response[0]
c_rate=response[1]

In [4]:
metrics_df=pd.read_csv('../data/results/Vanilla_FullPlanGeneration_Val_3_blocks')

In [8]:
print(metrics_df.to_markdown())

|    | Unnamed: 0              | Attempts   |   Structured |   Diff in plan length |   Terminate |   Action-steps away from goal |   Action-steps in GOLD standard |
|---:|:------------------------|:-----------|-------------:|----------------------:|------------:|------------------------------:|--------------------------------:|
|  0 | 0                       | 1          |            1 |                  -inf |           0 |                             4 |                               4 |
|  1 | 1                       | 1          |            1 |                  -inf |           0 |                             2 |                               6 |
|  2 | 2                       | 1          |            1 |                  -inf |           0 |                             4 |                               6 |
|  3 | 3                       | 1          |            1 |                  -inf |           0 |                             4 |                               4 |
|  4 | 4  

In [5]:
print('----------------GEMMA-2 9B, w demo help, w plan-end tags----------')
metrics_df

----------------GEMMA-2 9B, w demo help, w plan-end tags----------


,Unnamed: 0,Attempts,Structured,Diff in plan length,Terminate,Action-steps away from goal,Action-steps in GOLD standard
0,0,1,True,-inf,False,4.0,4.0
1,1,1,True,-inf,False,2.0,6.0
2,2,1,True,-inf,False,4.0,6.0
3,3,1,True,-inf,False,4.0,4.0
4,4,1,True,-inf,False,2.0,4.0
5,5,1,True,-inf,False,4.0,4.0
6,6,2,True,-inf,False,6.0,4.0
7,7,1,True,-inf,False,0.0,6.0
8,8,1,True,-inf,False,4.0,4.0
9,9,1,True,-inf,False,6.0,2.0


In [128]:
print(f'Correctness rate : {c_rate}')
print(f"Goal State Reached: {sum(metrics_df['Distance from goal']==0)}")

Correctness rate : 0.20
Goal State Reached: 3


In [330]:
metrics_df.to_csv(f'../data/results/Vanilla_FullPlanGeneration_Val_3_blocks')

In [20]:
problem=bw.BlocksworldProblem()

In [38]:
BLOCK='green'
blocks={'green':'green'}
block=blocks.get(BLOCK)
block

'green'

In [90]:
len(df)

15

In [ ]:
#qwen
Qwen/Qwen2-7B-Instruct
Qwen/Qwen2.5-7B-Instruct

#llama
meta-llama/Meta-Llama-3-8B-Instruct
meta-llama/Meta-Llama-3.1-8B-Instruct
meta-llama/Meta-Llama-3.1-8B-Instruct-Turbo

#mistral
mistralai/Mistral-7B-Instruct-v0.1
mistralai/Mistral-7B-Instruct-v0.2
mistralai/Mistral-7B-Instruct-v0.3

#phi-4
microsoft/Phi-4-multimodal-instruct

#gemma
google/gemma-1.1-7b-it
google/gemma-3-4b-it
google/gemma-2-9b-it